In [1]:
import os
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def local_registration_multiscale(source, target, init_transformation, voxel_sizes=[0.2, 0.1, 0.05]):
    """
    Multi-scale Colored ICP refinement.
    """
    current_transformation = init_transformation

    for i, voxel_size in enumerate(voxel_sizes):
        print(f"\nRefinement stage {i+1} with voxel_size={voxel_size} ...")
        distance_threshold = voxel_size * 1.0

        # Downsample for this scale
        source_down = source.voxel_down_sample(voxel_size)
        target_down = target.voxel_down_sample(voxel_size)

        # Estimate normals
        radius_normal = voxel_size * 2.0
        source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
        target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))

        # Colored ICP
        result = o3d.pipelines.registration.registration_icp(
            source_down, target_down, distance_threshold, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
        )

        print(f"  Fitness: {result.fitness:.4f}, Inlier RMSE: {result.inlier_rmse:.4f}")
        current_transformation = result.transformation

    return current_transformation

In [3]:
%cd pc

/csghome/sm330/pc


In [4]:
# Load point clouds
print("Loading point clouds...")
source_path = r"data/20240607_ground.ply"
target_path = r"data/20250613_ground.ply"

source = o3d.io.read_point_cloud(source_path, format='ply')
target = o3d.io.read_point_cloud(target_path, format='ply')

print("Original Source Points:", len(source.points))
print("Original Target Points:", len(target.points))

Loading point clouds...
Original Source Points: 13021998
Original Target Points: 18114341


In [5]:
# Start with identity transformation (already coarsely aligned)
initial_transformation = np.eye(4)  # Identity matrix - assuming clouds are already coarsely aligned

In [6]:
voxel_size = 0.1

In [7]:
print(f"\nRefinement with voxel_size={voxel_size}")
distance_threshold = voxel_size * 1.0

# Downsample for this scale
source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

# Estimate normals
radius_normal = voxel_size * 2.0
source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))


Refinement with voxel_size=0.1


In [8]:
result = o3d.pipelines.registration.registration_icp(
    source_down, target_down, distance_threshold, initial_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
)

In [9]:
print(f"  Fitness: {result.fitness:.4f}, Inlier RMSE: {result.inlier_rmse:.4f}")
current_transformation = result.transformation

  Fitness: 0.4293, Inlier RMSE: 0.0590


In [10]:
current_transformation

array([[ 9.99999999e-01, -3.69772167e-05,  3.55683298e-05,
         2.02124731e+02],
       [ 3.69743367e-05,  9.99999996e-01,  8.09693088e-05,
        -1.75559150e+01],
       [-3.55713237e-05, -8.09679935e-05,  9.99999996e-01,
         4.59353252e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [11]:
voxel_size = 0.05
print(f"\nRefinement with voxel_size={voxel_size}")
distance_threshold = voxel_size * 1.0

# Downsample for this scale
source_down = source.voxel_down_sample(voxel_size)
target_down = target.voxel_down_sample(voxel_size)

# Estimate normals
radius_normal = voxel_size * 2.0
source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=50))
result = o3d.pipelines.registration.registration_icp(
    source_down, target_down, distance_threshold, initial_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
)
print(f"  Fitness: {result.fitness:.4f}, Inlier RMSE: {result.inlier_rmse:.4f}")
current_transformation_005 = result.transformation


Refinement with voxel_size=0.05
  Fitness: 0.2668, Inlier RMSE: 0.0336
